<a href="https://colab.research.google.com/github/AnnJungChan/youngjin_TECH/blob/main/DeepCandle/no1_candle_chart_data_diff_30minutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지로딩

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler

Mounted at /content/gdrive


# 데이터 로딩

- 1호기 대상으로 실행한다.
- 현재 1호기 데이터는 PCA를 진행한 후, 차분이 된 데이터이다.

In [2]:
data=pd.read_csv("/content/gdrive/MyDrive/용접데이터/diff_data.csv")
data.drop(columns = ['Unnamed: 0'],inplace = True)
data

,date,time,diff1
0,2021-01-04,08:40:09,NaN
1,2021-01-04,08:40:29,-0.000855
2,2021-01-04,08:40:44,-0.000295
3,2021-01-04,08:40:59,-0.194407
4,2021-01-04,08:41:15,0.197794
...,...,...,...
302964,2021-10-28,17:50:20,-0.001370
302965,2021-10-28,17:50:34,0.001517
302966,2021-10-28,17:50:49,0.000589
302967,2021-10-28,17:51:04,0.098559


# 캔들차트를 위한 데이터 전처리

- 캔들차트를 그리기 위해서 1시간 간격으로 데이터를 전처리함.

## 날짜 계산

- 날짜를 정확히 계산하여, 1시간 간격으로 데이터를 자르고 전처리할 준비를 함.

In [3]:
data=data[(data['date']>='2021-01-05')&(data['date']<='2021-10-28')]
data

,date,time,diff1
907,2021-01-05,08:39:25,0.093714
908,2021-01-05,08:39:49,0.003976
909,2021-01-05,08:40:04,0.003682
910,2021-01-05,08:40:19,-0.001031
911,2021-01-05,08:40:34,-0.000589
...,...,...,...
302964,2021-10-28,17:50:20,-0.001370
302965,2021-10-28,17:50:34,0.001517
302966,2021-10-28,17:50:49,0.000589
302967,2021-10-28,17:51:04,0.098559


In [4]:
print('최소날짜 : ', data['date'].min())
print('최대날짜 : ', data['date'].max())
print('최소시간 : ' , data['time'].min())
print('최대시간 : ' , data['time'].max())

최소날짜 :  2021-01-05
최대날짜 :  2021-10-28
최소시간 :  07:48:35
최대시간 :  20:55:37


In [ ]:
import datetime
import time

time = datetime.datetime(2021,1,5,8,30,0)
print("날짜 차이 : " ,datetime.datetime(2021,1,5) - datetime.datetime(2021,10,28))

days = []
days.append(time)
for i in range(296) : 
  days.append(days[i] + datetime.timedelta(days = 1))

In [ ]:
start_days = days
end_days = []
for i in range(len(days)) :
  end_days.append(days[i] + datetime.timedelta(hours = 14))

In [41]:
time_min = []
time_diff = np.arange(0,900,30,dtype = 'float')
for j in range(len(start_days)) :
  for i in range(len(time_diff)) :
    time_min.append(start_days[j] + datetime.timedelta(minutes= time_diff[i]))

시간은 30분 단위로 쪼개서 만들었음

In [ ]:
time_str = []
for i in range(len(time_min)) :
  time_str.append(time_min[i].strftime('%Y-%m-%d %H:%M:%S'))

time_str이 시간 축이 되어 left로 데이터를 붙이는 역할을 하고, 값이 비었다면 time_str값을 제거시켜주면서 진행할 것이다.

## DIFF1에서 진행

In [48]:
data['date_time'] = data['date'] + " " + data['time']

,date,time,diff1,date_time
907,2021-01-05,08:39:25,0.093714,2021-01-05 08:39:25
908,2021-01-05,08:39:49,0.003976,2021-01-05 08:39:49
909,2021-01-05,08:40:04,0.003682,2021-01-05 08:40:04
910,2021-01-05,08:40:19,-0.001031,2021-01-05 08:40:19
911,2021-01-05,08:40:34,-0.000589,2021-01-05 08:40:34
...,...,...,...,...
302964,2021-10-28,17:50:20,-0.001370,2021-10-28 17:50:20
302965,2021-10-28,17:50:34,0.001517,2021-10-28 17:50:34
302966,2021-10-28,17:50:49,0.000589,2021-10-28 17:50:49
302967,2021-10-28,17:51:04,0.098559,2021-10-28 17:51:04


In [ ]:
top = []
bottom = []
min = []
max = []
volume = []


time = []

for i in range(len(time_str)) :
  temp=data.loc[(time_str[i]<=data['date_time']) & (time_str[i+1]>=data['date_time']),"diff1"]
  if len(temp) != 0 :
    time.append(time_str[i])


for i in range(len(time)) :
  temp=data.loc[(time[i]<=data['date_time']) & (time[i+1]>=data['date_time']),"diff1"]
  top.append(temp.head(1).values)
  bottom.append(temp.tail(1).values)
  min.append(temp.min())
  max.append(temp.max())
  volume.append(len(temp))

In [52]:
print("time 길이 : ", len(time))
print("top 길이 : ", len(top))
print("bottom 길이 : ", len(bottom))
print("min 길이 : ", len(min))
print("max 길이 : ", len(max))
print("volume 길이 : ", len(volume))

time 길이 :  3789
top 길이 :  3788
bottom 길이 :  3788
min 길이 :  3788
max 길이 :  3788
volume 길이 :  3788


In [53]:
time = time[:3788]
data=pd.DataFrame({'time' : time, 'top' : top, 'bottom' : bottom, "min" : min, "max" : max, "volume" : volume })

In [54]:
data['top']=data['top'].astype(float)
data['bottom']=data['bottom'].astype(float)
data['min']=data['min'].astype(float)
data['max']=data['max'].astype(float)
data['volume']=data['volume'].astype(int)
data['time'] = data['time'].astype('datetime64')
data.to_csv("/content/gdrive/MyDrive/용접데이터/캔들차트분석/1호기 프로세스1/30min_diff1.csv")